In [77]:
import pandas as pd
import plotly.graph_objects as go

In [78]:
df_en=pd.read_csv('all_communities_infomap.csv',sep=";")
df_en=df_en[df_en['Year']==2024]
df_other=pd.read_csv('all_communities_infomap_de.csv',sep=";")
df_other=df_other[df_other['Year']==2024]


In [79]:
lang='De'
df_other['Members']=df_other['Members'].apply(lambda x: x.split(', '))
df_other=df_other.explode('Members')
df_other.rename(columns = {'Members':'Name'}, inplace = True)
df_en['Members']=df_en['Members'].apply(lambda x: x.split(', '))
df_en=df_en.explode('Members')
df_en.rename(columns = {'Members':'Name'}, inplace = True)

In [80]:
en_names=df_en['Name'].tolist()
other_names=df_other['Name'].tolist()
common_names=list(set(en_names).intersection(other_names))

In [81]:
size=50

In [82]:
df_en=df_en[df_en['Name'].isin(common_names)]
df_en['Size']=df_en.groupby('Community ID')['Community ID'].transform('count')
df_other=df_other[df_other['Name'].isin(common_names)]
df_other['Size']=df_other.groupby('Community ID')['Community ID'].transform('count')

In [83]:
df_en.loc[df_en['Size']<=size,'Community ID']='Other'
df_other.loc[df_other['Size']<=size,'Community ID']='Other'

df_en=df_en.drop_duplicates(subset=['Name','Community ID'])
df_other=df_other.drop_duplicates(subset=['Name','Community ID'])



In [84]:
df_en=df_en[['Name','Community ID']]
df_other=df_other[['Name','Community ID']]

df_en=df_en.rename(columns={'Community ID':'CommunityID'})
df_other=df_other.rename(columns={'Community ID':'CommunityID'})

df_en['CommunityID']=df_en['CommunityID'].astype(str)
df_other['CommunityID']=df_other['CommunityID'].astype(str)



In [85]:
df= pd.concat([
    df_en.assign(Lang='En'),
    df_other.assign(Lang=lang)
])

In [86]:
df_pivot = df.pivot_table(index="Name", columns="Lang", values="CommunityID", aggfunc='first')
df_pivot = df_pivot[['En',lang]]
df_combined = df_pivot.reset_index()




In [87]:
colorlist=['rgb(3,29,68)','rgb(132,5,251)','rgb(175,94,192)','rgb(109,190,152)', 'rgb(0,196,170)','rgb(67,100,54)']
neutralcolor='rgb(218,183,133)'

In [88]:
df_filtered=df_combined
top_6=df_filtered['En'].value_counts().index[:6]



dim_other = go.parcats.Dimension(
    values=df_filtered.De,
    label="German",
    categoryorder='array',
    categoryarray=[x for x in df_filtered.De.unique() if x != 'Other'] + ['Other']
)
dim_en = go.parcats.Dimension(
    values=df_filtered.En,
    label="English",
    categoryorder='array',
    categoryarray=[x for x in df_filtered.En.unique() if x != 'Other'] + ['Other']
)
dim=[dim_other,dim_en]
color = df_filtered.En.apply(lambda x: '#031D44' if x == '3' else '#8405FB' if x == '1' else ('#9A32DE' if x=='6' else( '#AF5EC0' if x=='4' else( '#C58BA3' if x=='10' else('#B5A399' if x=='7' else('#A4BB8F' if x=='13' else('#6DBE98' if x=='2' else('#00C4AA' if x=='5' else('#436436' if x=='8' else('#007A6A' if x=='11' else('#0853C4' if x=='9' else neutralcolor)))))))))))
colorscale = [[0, neutralcolor], [0.5, neutralcolor], [1, neutralcolor]];
fig = go.Figure(data = [go.Parcats(dimensions=dim,
        line={'color': color, 'colorscale': colorscale,'shape': 'hspline'},
        hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])
fig.update_layout(
    margin=dict(l=10, r=10, t=20, b=10),
    height=1500,
    width=800,
    title_x=0.5
)
fig.show()
Language='German'
size=str(size)
fig.write_html("Sankey/"+Language+" (minsize=" + size + ").html")
